In [25]:
# Task parser
from datetime import datetime
import csv

file_path = r'data_t1/data_for_spb_hakaton_entities1-Table 1.csv'

with open(file_path, newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=';', quotechar='"')
    task_data = [row for row in csv_reader]

# print(task_data)

In [26]:
# History parser

def add_or_update_key(dictionary : dict, key, value_to_add):
    dictionary[key] = dictionary.setdefault(key, [])
    dictionary[key].append(value_to_add)

file_path2 = r'data_t1/history-Table 1.csv'

with open(file_path2, newline='', encoding='utf-8') as csvfile:
    csv_reader2 = csv.reader(csvfile, delimiter=';', quotechar='"')
    history_data = [row for row in csv_reader2]

history_main_dict = {}
unique_events = set()

for pack in history_data:
    add_or_update_key(history_main_dict, pack[0], pack[1:])
    if pack[1] == 'Резолюция':
        unique_events.add(pack[5])
print(history_main_dict['4337639'])
print(unique_events)

[['Дата решения', '7/26/24 6:00', '1', 'FIELD_CHANGED', '<empty> -> 2024-07-23T07:35:24+00:00', '', ''], ['Задача', '6/24/24 11:53', '1', 'CREATED', '', '', ''], ['Связанные Задачи', '6/24/24 11:53', '2', 'FIELD_CHANGED', '<empty> -> isClonedBy', '', ''], ['Связанные Задачи', '6/24/24 11:53', '2', 'FIELD_CHANGED', '<empty> -> PPTS-7551', '', ''], ['Связанные Задачи', '6/24/24 11:53', '3', 'FIELD_CHANGED', '', '', ''], ['Согласие на передачу прав', '6/24/24 11:53', '4', 'FIELD_CHANGED', '<empty> -> true', '', ''], ['Связанные Задачи', '6/24/24 11:55', '5', 'FIELD_CHANGED', '<empty> -> clones', '', ''], ['Связанные Задачи', '6/24/24 11:55', '5', 'FIELD_CHANGED', '<empty> -> PPTS-7554', '', ''], ['Связанные Задачи', '6/24/24 11:55', '6', 'FIELD_CHANGED', '', '', ''], ['Связанные Задачи', '6/24/24 11:56', '7', 'FIELD_CHANGED', '<empty> -> clones', '', ''], ['Связанные Задачи', '6/24/24 11:56', '7', 'FIELD_CHANGED', '<empty> -> PPTS-7555', '', ''], ['Связанные Задачи', '6/24/24 11:56', '8',

In [27]:
# sprints-table parser



file_path2 = r'data_t1/sprints-Table 1.csv'

with open(file_path2, newline='', encoding='utf-8') as csvfile:
    csv_reader2 = csv.reader(csvfile, delimiter=';', quotechar='"')
    sprint_data = [row for row in csv_reader2]

# for i in sprint_data:
#     print(i)

In [28]:
# investigate one sprint ! Спринт 2024.3.1.NPP Shared Sprint

def search_through_status_changes(key, fake_status_changes):
    status_change_trace = []
    for event in history_main_dict[key]:
        #! print(delta_time := ((datetime.strptime(sprint_end_date, '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(event[1], '%m/%d/%y %H:%M')).seconds / 3600))
        if event[0] == "Статус":
            status_change_trace.append(datetime.strptime(event[1], '%m/%d/%y %H:%M'))
    status_change_trace.sort()
    time_diference = []
    for i in range(1, len(status_change_trace)):
        time_diference.append((status_change_trace[i] - status_change_trace[i - 1]).total_seconds() / 60)
    if len(time_diference) != 0 and (sum(time_diference) / len(time_diference)) <= 5:
        fake_status_changes += 1
        
    print(key, time_diference, status_change_trace)
    return fake_status_changes
        
def search_for_last_day_status_change(key, sprint_end_date, last_day_completed_counter):
    for event in history_main_dict[key]:
        if (event[0] == "Статус") and (event[4].split(' ')[-1] == 'closed'):
           delta_time = ((datetime.strptime(sprint_end_date, '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(event[1], '%m/%d/%y %H:%M'))).days
           if delta_time == 0:
               print(key, event)
               last_day_completed_counter += 1
    return last_day_completed_counter

for pack in sprint_data:
    if 'Спринт 2024.3.2.NPP Shared Sprint' in pack:
        sprint_end_date = pack[3]
        sprint_start_date = pack[2]
        enteties = pack[-1]
        enteties = enteties.replace('{', '')
        enteties = enteties.replace('}', '')
        entitiences = enteties.split(",")
        # print(entitiences)
        # print(sprint_end_date)
        break

fake_changes = 0
last_day_completions = 0
for instance in entitiences:
    fake_changes = search_through_status_changes(instance, fake_changes)
    last_day_completions = search_for_last_day_status_change(instance, sprint_end_date, last_day_completions)

print(fake_changes, fake_changes_percent := round((fake_changes / len(entitiences)) * 100) )
print(last_day_completions, last_day_completions_percent := round((last_day_completions / len(entitiences)) * 100) )






4506286 [81.0, 0.0, 2.0] [datetime.datetime(2024, 7, 17, 9, 19), datetime.datetime(2024, 7, 17, 10, 40), datetime.datetime(2024, 7, 17, 10, 40), datetime.datetime(2024, 7, 17, 10, 42)]
4429413 [0.0, 7555.0] [datetime.datetime(2024, 7, 25, 11, 43), datetime.datetime(2024, 7, 25, 11, 43), datetime.datetime(2024, 7, 30, 17, 38)]
4429413 ['Статус', '7/30/24 17:38', '14', 'FIELD_CHANGED', 'done -> closed', '', '']
4327418 [] [datetime.datetime(2024, 7, 22, 7, 49)]
4370041 [0.0] [datetime.datetime(2024, 7, 19, 8, 12), datetime.datetime(2024, 7, 19, 8, 12)]
4525683 [16.0] [datetime.datetime(2024, 7, 22, 7, 13), datetime.datetime(2024, 7, 22, 7, 29)]
4326715 [] [datetime.datetime(2024, 7, 22, 6, 53)]
4509890 [15849.0] [datetime.datetime(2024, 7, 18, 4, 44), datetime.datetime(2024, 7, 29, 4, 53)]
4556454 [] [datetime.datetime(2024, 7, 30, 15, 21)]
4556454 ['Статус', '7/30/24 15:21', '8', 'FIELD_CHANGED', 'created -> closed', '', '']
3910459 [8549.0] [datetime.datetime(2024, 7, 24, 6, 18), datet

In [70]:
def according_to_datestamp(d2_array, datestamp):
    res = []
    for inst in d2_array:
        if datetime.strptime(inst[1], '%m/%d/%y %H:%M') <= datestamp:
            res.append(inst)
    return res

def universal_sprint_counting_mashine(sprint, datestamp):
    sucsess = fail = created = ongoing = 0
    for instance in entitiences:
        instance_data = according_to_datestamp(history_main_dict[instance], datestamp)
        sorted_instance_data = sorted(instance_data, key=lambda x: x[1])
        resolution = status = task = None
        for i in range(len(sorted_instance_data)):
            if sorted_instance_data[i][0] == 'Резолюция': resolution = sorted_instance_data[i][4].split(' ')[-1]
            if sorted_instance_data[i][0] == 'Статус': status = sorted_instance_data[i][4].split(' ')[-1]
            if sorted_instance_data[i][0] == 'Задача': status = sorted_instance_data[i][3].split(' ')[-1]
        # !print(status, resolution)
        if (status == 'closed' or status == 'done') and (resolution == 'Готово' or resolution == None):
            sucsess += 1
        elif (resolution in ['инициатором', 'Дубликат', 'Отклонено']):
            fail += 1 
        elif status == 'CREATED' or (status == None and resolution == None):
            created += 1
        elif status != 'closed' and status != 'CREATED':
            ongoing += 1
    print(distribution := list(map(lambda x: x / len(entitiences) * 100, [sucsess, fail, created, ongoing])))  
    return distribution
    

        
distribution = list(map(round, universal_sprint_counting_mashine(1, datetime.strptime('2024-07-25 19:00:00.000000', '%Y-%m-%d %H:%M:%S.%f'))))

[21.73913043478261, 3.4324942791762014, 37.07093821510298, 37.75743707093822]


In [30]:
date_string = '7/16/24 8:21'
date_string2 = '2024-07-16 19:00:00.000000'
parsed_date = datetime.strptime(date_string, '%m/%d/%y %H:%M')
parsed_date2 = datetime.strptime(date_string2, '%Y-%m-%d %H:%M:%S.%f')
x = [parsed_date2, parsed_date]
x.sort()
print(x)

[datetime.datetime(2024, 7, 16, 8, 21), datetime.datetime(2024, 7, 16, 19, 0)]


In [71]:
sprint_health_dict = {'irrelevant_status_track_tasks': f"{fake_changes_percent}%",
                      'last_day_completed_status_tasks': f"{last_day_completions_percent}%",
                      'tasks_in_work' : f"{distribution[3]}%",
                      'tasks_in_creation': f"{distribution[2]}%",
                      'tasks_done' : f"{distribution[0]}%",
                      'tasks_canceled' : f"{distribution[1]}%",
                   }

print(sprint_health_dict)

{'irrelevant_status_track_tasks': '12%', 'last_day_completed_status_tasks': '23%', 'tasks_in_work': '38%', 'tasks_in_creation': '37%', 'tasks_done': '22%', 'tasks_canceled': '3%'}
